**DEEP LEARNING**

YOLOV3 - Object recognition

GABRIEL JOSHUA . R

### Importing the necessary dependencies

In [ ]:
import cv2 as cv
import numpy as np
from google.colab.patches import cv2_imshow

### Mount Google Drive to access the required files

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

### Load the image from Google Drive and display it

In [ ]:
img = cv.imread('/gdrive/My Drive/DEEPLEARNING/dogs.jpg')
cv2_imshow(img)

### Preprocess the image using YOLO's blobFromImage function


In [ ]:
blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
r = blob[0, 0, :, :]

### Create a copy of the original image for visualization purposes


In [ ]:
imgtemp = img.copy()

### Load the names of classes and assign random colors to each class


In [ ]:
classes = open('/gdrive/My Drive/DEEPLEARNING/coco.names').read().strip().split('\n')
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classes), 3), dtype='uint8')

### Load the YOLO model using the configuration and weight files


In [ ]:
net = cv.dnn.readNetFromDarknet('/gdrive/My Drive/DEEPLEARNING/yolov3.cfg', '/gdrive/My Drive/DEEPLEARNING/yolov3.weights')
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
CON = 0.7

### Determine the output layer names

In [ ]:
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

### Set the blob as the input to the network and get the outputs


In [ ]:
net.setInput(blob)
outputs = net.forward(ln)

### Function to update the confidence threshold using a trackbar


In [ ]:
def trackbar2(x):
    confidence = x / 100
    r = r0.copy()
    for output in np.vstack(outputs):
        if output[4] > confidence:
            x, y, w, h = output[:4]
            p0 = int((x - w/2) * 416), int((y - h/2) * 416)
            p1 = int((x + w/2) * 416), int((y + h/2) * 416)
            cv.rectangle(r, p0, p1, 1, 1)
            cv.rectangle(imgtemp, p0, p1, 1, 1)
    w1, h1 = imgtemp.shape[:2]
    print("height:", h1, "width:", w1)
    cv2_imshow(imgtemp)

### Get the initial image for the trackbar function

In [ ]:
r0 = blob[0, 0, :, :]
r = r0.copy()
trackbar2(CON*100)

### Extract bounding boxes, confidences, and class IDs from the outputs


In [ ]:
boxes = []
confidences = []
classIDs = []
h, w = img.shape[:2]
print("height:", h, "width:", w)
for output in outputs:
    for detection in output:
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        if confidence > CON:
            box = detection[:4] * np.array([w, h, w, h])
            (centerX, centerY, width, height) = box.astype("int")
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            box = [x, y, int(width), int(height)]
            boxes.append(box)
            confidences.append(float(confidence))
            classIDs.append(classID)

### Non-maximum suppression to eliminate redundant detections


In [ ]:
score_threshold = 0.5
nms_threshold = 0.5

In [ ]:
print("score threshold", score_threshold, "nms Threshold", nms_threshold)

indices = cv.dnn.NMSBoxes(boxes, confidences, score_threshold, nms_threshold)

### Draw the final bounding boxes and labels on the image


In [ ]:
if len(indices) > 0:
    for i in indices.flatten():
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        color = [int(c) for c in colors[classIDs[i]]]
        cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
        text = "{}: {:.4f}".format(classes[classIDs[i]], confidences[i])
        cv.putText(img, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

cv2_imshow(img)

### Variables and data you want to access outside the code block


In [ ]:
text

In [ ]:
classes

In [ ]:
print(len(classes))